<a href="https://colab.research.google.com/github/YG15/DataHack2019/blob/master/yonathan_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from google.colab import files

In [3]:
!wget https://raw.githubusercontent.com/YG15/DataHack2019/master/src/functions/get_network_hierarchy.py

from get_network_hierarchy import *

--2019-09-04 20:59:19--  https://raw.githubusercontent.com/YG15/DataHack2019/master/src/functions/get_network_hierarchy.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1703 (1.7K) [text/plain]
Saving to: ‘get_network_hierarchy.py.1’

get_network_hierarc 100%[===================>]   1.66K  --.-KB/s    in 0s      

2019-09-04 20:59:20 (329 MB/s) - ‘get_network_hierarchy.py.1’ saved [1703/1703]



In [4]:
url = 'https://armis-datahack.s3.amazonaws.com/all_devices.csv'
devices_df = pd.read_csv(url, index_col = 0)
devices_df.head()

,network_id,device_id,type,model,manufacturer,operating_system,operating_system_version
113,0,1028623,MOBILE_PHONE,Galaxy S8,Samsung,Android,9
587,0,48047,MOBILE_PHONE,Galaxy Note 8,Samsung,Android,9
668,0,123568,MOBILE_PHONE,H918,LG Electronics,Android,8.0.0
830,0,95366,MOBILE_PHONE,iPhone 6,"Apple, Inc.",iOS,NaN
886,0,1755023,TABLET,iPad,Apple,iOS,NaN


In [5]:
url = 'https://armis-datahack.s3.amazonaws.com/all_sessions.csv'
sessions_df = pd.read_csv(url)
sessions_df.head()

,Unnamed: 0,network_id,device_id,timestamp,host,host_ip,port_dst,transport_protocol,service_device_id,packets_count,outbound_bytes_count,inbound_bytes_count,packet_loss,retransmit_count,latency,session_count,outbound_packets_count,inbound_packets_count,outbound_bytes_max,outbound_bytes_min,outbound_bytes_mean,outbound_bytes_median,outbound_bytes_stddev,inbound_bytes_max,inbound_bytes_min,inbound_bytes_mean,inbound_bytes_median,inbound_bytes_stddev,outbound_packet_size_max,outbound_packet_size_min,outbound_packet_size_mean,outbound_packet_size_median,outbound_packet_size_stddev,inbound_packet_size_max,inbound_packet_size_min,inbound_packet_size_mean,inbound_packet_size_median,inbound_packet_size_stddev
0,0,0,35,1565074800,ecbb92cd941972b779d18451b6f96275587941e4cf07a1...,ecbb92cd941972b779d18451b6f96275587941e4cf07a1...,49152,TCP,790889.0,260,21855,7815,0.067503,67,NaN,39,235,25,930,465,560.384615,NaN,190.219349,1563,0,200.384615,NaN,529.370014,93.000000,93.000000,93.000000,93.000000,0.000000,312.6,312.6,312.6,312.6,0.0
1,1,0,35,1565053200,90cf529b11c8f26efbb3936c7d10a5bf57c1a930603af0...,90cf529b11c8f26efbb3936c7d10a5bf57c1a930603af0...,49153,TCP,1604622.0,178,8326,28656,0.000000,16,NaN,18,88,90,467,427,462.555556,467.0,12.935233,1592,1592,1592.000000,1592.0,0.000000,106.750000,93.400000,94.883333,93.000000,4.317134,318.4,318.4,318.4,318.0,0.0
2,2,0,57,1565082000,e16257c983f2c35d41f39d425651972fa1905e46e968d7...,d43dad76e6cef2231d2efc743e498996b40f8b13fc120b...,443,TCP,NaN,67,4013,0,0.000000,40,NaN,7,67,0,3521,82,573.285714,NaN,1299.819823,0,0,0.000000,NaN,0.000000,64.018182,41.000000,44.288312,41.000000,8.700055,NaN,NaN,NaN,NaN,NaN
3,3,0,57,1565082000,1a4f860269acca6c264f00d84c4b63aad00b8f93a77250...,945e37dab8aee93dd4e650f8d17d76a3adfbc6aa70ebba...,443,TCP,NaN,45,9158,0,0.000000,3,NaN,4,45,0,3401,935,2289.500000,2411.0,1018.074490,0,0,0.000000,0.0,0.000000,226.733333,133.571429,194.968236,209.000000,41.777266,NaN,NaN,NaN,NaN,NaN
4,4,0,57,1565082000,df106cbe1ba4a700c00ec8883490f40a8afdb75c15a9ea...,4b43e85e630c2d18a0afaa2a6366367c4fc52d32b4ba5b...,443,TCP,NaN,9,1671,0,0.000000,0,NaN,1,9,0,1671,1671,1671.000000,NaN,NaN,0,0,0.000000,NaN,NaN,185.666667,185.666667,185.666667,185.666667,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#df_processed = get_network_hierarchy(df)

In [0]:
def range_list(val_list):
    min_val = min(val_list)
    max_val = max(val_list)

    return (max_val-min_val)
  
classic_functions = ['min','max','mean','std']

df_gb = sessions_df.groupby(['device_id'],as_index=False).agg({'host':'nunique',
                                                      'host_ip':'nunique',
                                                      'port_dst':'nunique',
                                                      'timestamp':['nunique','count',('range', lambda x: range_list(x))],
                                                      'transport_protocol':'nunique',
                                                      'packets_count':classic_functions,
                                                      'outbound_bytes_count':classic_functions, 
                                                      'inbound_bytes_count':classic_functions, 
                                                      'packet_loss':classic_functions,
                                                      'retransmit_count':classic_functions, 
                                                      'latency':classic_functions, 
                                                      'session_count':classic_functions,
                                                      'outbound_packets_count':classic_functions, 
                                                      'inbound_packets_count':classic_functions, 
                                                      'outbound_bytes_max':classic_functions,
                                                      'outbound_bytes_min':classic_functions, 
                                                      'outbound_bytes_mean':classic_functions, 
                                                      'outbound_bytes_median':classic_functions,
                                                      'outbound_bytes_stddev':classic_functions, 
                                                      'inbound_bytes_max':classic_functions, 
                                                      'inbound_bytes_min':classic_functions,
                                                      'inbound_bytes_mean':classic_functions, 
                                                      'inbound_bytes_median':classic_functions, 
                                                      'inbound_bytes_stddev':classic_functions,
                                                      'outbound_packet_size_max':classic_functions, 
                                                      'outbound_packet_size_min':classic_functions,
                                                      'outbound_packet_size_mean':classic_functions, 
                                                      'outbound_packet_size_median':classic_functions,
                                                      'outbound_packet_size_stddev':classic_functions, 
                                                      'inbound_packet_size_max':classic_functions,
                                                      'inbound_packet_size_min':classic_functions, 
                                                      'inbound_packet_size_mean':classic_functions,
                                                      'inbound_packet_size_median':classic_functions, 
                                                      'inbound_packet_size_stddev':classic_functions})

df_gb.columns = ["_".join(x) for x in df_gb.columns.ravel()]

In [9]:
df_gb.sample(20)

,device_id_,host_nunique,host_ip_nunique,port_dst_nunique,timestamp_nunique,timestamp_count,timestamp_range,transport_protocol_nunique,packets_count_min,packets_count_max,packets_count_mean,packets_count_std,outbound_bytes_count_min,outbound_bytes_count_max,outbound_bytes_count_mean,outbound_bytes_count_std,inbound_bytes_count_min,inbound_bytes_count_max,inbound_bytes_count_mean,inbound_bytes_count_std,packet_loss_min,packet_loss_max,packet_loss_mean,packet_loss_std,retransmit_count_min,retransmit_count_max,retransmit_count_mean,retransmit_count_std,latency_min,latency_max,latency_mean,latency_std,session_count_min,session_count_max,session_count_mean,session_count_std,outbound_packets_count_min,outbound_packets_count_max,outbound_packets_count_mean,outbound_packets_count_std,...,outbound_packet_size_max_min,outbound_packet_size_max_max,outbound_packet_size_max_mean,outbound_packet_size_max_std,outbound_packet_size_min_min,outbound_packet_size_min_max,outbound_packet_size_min_mean,outbound_packet_size_min_std,outbound_packet_size_mean_min,outbound_packet_size_mean_max,outbound_packet_size_mean_mean,outbound_packet_size_mean_std,outbound_packet_size_median_min,outbound_packet_size_median_max,outbound_packet_size_median_mean,outbound_packet_size_median_std,outbound_packet_size_stddev_min,outbound_packet_size_stddev_max,outbound_packet_size_stddev_mean,outbound_packet_size_stddev_std,inbound_packet_size_max_min,inbound_packet_size_max_max,inbound_packet_size_max_mean,inbound_packet_size_max_std,inbound_packet_size_min_min,inbound_packet_size_min_max,inbound_packet_size_min_mean,inbound_packet_size_min_std,inbound_packet_size_mean_min,inbound_packet_size_mean_max,inbound_packet_size_mean_mean,inbound_packet_size_mean_std,inbound_packet_size_median_min,inbound_packet_size_median_max,inbound_packet_size_median_mean,inbound_packet_size_median_std,inbound_packet_size_stddev_min,inbound_packet_size_stddev_max,inbound_packet_size_stddev_mean,inbound_packet_size_stddev_std
30656,126154,2,2,1,1,2,0,1,25,115,70.000000,63.639610,3991,4613,4302.000000,4.398204e+02,5937,65501,3.571900e+04,4.211811e+04,0.000000,0.000000,0.000000,0.000000,0,6,3.000000,4.242641,NaN,NaN,NaN,NaN,1,1,1.000000,0.000000,13,54,33.500000,28.991378,...,85.425926,307.000000,196.212963,156.676530,85.425926,307.000000,196.212963,156.676530,85.425926,307.000000,196.212963,156.676530,85.425926,307.000000,196.212963,156.676530,NaN,NaN,NaN,NaN,494.750000,1073.786885,784.268443,409.440908,494.750000,1073.786885,784.268443,409.440908,494.750000,1073.786885,784.268443,409.440908,494.750000,1073.786885,784.268443,409.440908,NaN,NaN,NaN,NaN
59714,702360,6,5,3,1,6,0,2,2,19,8.333333,5.921711,130,3919,2284.833333,1.628373e+03,0,0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN,NaN,NaN,1,2,1.166667,0.408248,2,19,8.333333,5.921711,...,68.000000,434.777778,252.546505,128.374550,62.000000,434.777778,251.546505,130.111247,65.000000,434.777778,252.046505,129.240013,65.000000,434.777778,252.046505,129.240013,4.242641,4.242641,4.242641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55050,530682,70,83,11,3,128,7200,2,4,25240,493.828125,2267.088903,176,1136896,46435.343750,1.378124e+05,0,14392352,1.560790e+05,1.274628e+06,0.000000,0.901914,0.728328,0.142646,0,18912,362.101562,1700.601412,1.0,416.0,108.000000,143.709777,1,29,2.453125,4.301363,4,15388,325.765625,1399.161523,...,44.000000,986.250000,186.096540,193.016717,44.000000,833.666667,133.786497,139.877222,44.000000,833.666667,157.629695,152.977423,44.000000,833.666667,156.178837,153.564821,0.000000,482.946246,73.565670,116.502665,40.000000,1532.190260,417.855220,408.212323,40.000000,1434.578744,348.130851,393.478539,40.000000,1497.514306,380.070734,395.025352,40.000000,1516.505092,376.371092,397.806708,0.000000,529.622979,109.321895,128.216799
26675,100963,17,42,3,2,52,172800,2,5,301,39.423077,58.105160,495,39812,4589.500000,7.632398e+03,0,6464,4.679038e+02,1.

In [11]:
final_df = pd.merge(df_gb, devices_df, how = 'left', left_on='device_id_' , right_on='device_id')
final_df.head()

,device_id_,host_nunique,host_ip_nunique,port_dst_nunique,timestamp_nunique,timestamp_count,timestamp_range,transport_protocol_nunique,packets_count_min,packets_count_max,packets_count_mean,packets_count_std,outbound_bytes_count_min,outbound_bytes_count_max,outbound_bytes_count_mean,outbound_bytes_count_std,inbound_bytes_count_min,inbound_bytes_count_max,inbound_bytes_count_mean,inbound_bytes_count_std,packet_loss_min,packet_loss_max,packet_loss_mean,packet_loss_std,retransmit_count_min,retransmit_count_max,retransmit_count_mean,retransmit_count_std,latency_min,latency_max,latency_mean,latency_std,session_count_min,session_count_max,session_count_mean,session_count_std,outbound_packets_count_min,outbound_packets_count_max,outbound_packets_count_mean,outbound_packets_count_std,...,outbound_packet_size_min_std,outbound_packet_size_mean_min,outbound_packet_size_mean_max,outbound_packet_size_mean_mean,outbound_packet_size_mean_std,outbound_packet_size_median_min,outbound_packet_size_median_max,outbound_packet_size_median_mean,outbound_packet_size_median_std,outbound_packet_size_stddev_min,outbound_packet_size_stddev_max,outbound_packet_size_stddev_mean,outbound_packet_size_stddev_std,inbound_packet_size_max_min,inbound_packet_size_max_max,inbound_packet_size_max_mean,inbound_packet_size_max_std,inbound_packet_size_min_min,inbound_packet_size_min_max,inbound_packet_size_min_mean,inbound_packet_size_min_std,inbound_packet_size_mean_min,inbound_packet_size_mean_max,inbound_packet_size_mean_mean,inbound_packet_size_mean_std,inbound_packet_size_median_min,inbound_packet_size_median_max,inbound_packet_size_median_mean,inbound_packet_size_median_std,inbound_packet_size_stddev_min,inbound_packet_size_stddev_max,inbound_packet_size_stddev_mean,inbound_packet_size_stddev_std,network_id,device_id,type,model,manufacturer,operating_system,operating_system_version
0,13,33,27,5,3,39,108000,2,1,7039,378.256410,1309.550741,63,412746,32313.564103,91017.710361,0,1074,64.128205,238.283133,0.000000,0.344403,0.018762,0.061293,0,393,19.692308,70.297546,NaN,NaN,NaN,NaN,1,14,3.205128,3.894527,1,7039,377.615385,1309.725516,...,28.765872,57.512597,350.262997,113.021300,58.988158,57.0,302.000000,109.250560,55.473809,2.436279,217.001429,69.516902,62.300135,54.666667,221.500000,163.655556,94.446577,54.666667,221.500000,163.655556,94.446577,54.666667,221.500000,163.655556,94.446577,54.666667,221.500000,163.655556,94.446577,NaN,NaN,NaN,NaN,2,13,MOBILE_PHONE,iPhone XR,"Apple, Inc.",iOS,12.3.1
1,20,139,199,55,26,656,205200,2,1,3600,75.306402,258.681010,44,351711,4453.356707,15721.064738,0,2141582,27769.682927,150525.579066,0.000000,0.711570,0.042561,0.112028,0,624,5.434451,27.470211,1.0,77.0,11.105263,20.177308,1,37,1.789634,2.584442,1,3117,41.974085,154.599332,...,73.816150,44.000000,439.457143,121.009445,74.935238,44.0,439.457143,120.695290,75.472533,0.000000,370.700730,23.183979,48.010039,44.000000,1278.300317,430.885295,313.295112,40.000000,1278.300317,356.217065,304.084296,44.000000,1278.300317,393.970049,291.893607,40.000000,1278.300317,394.473242,294.911721,0.00000,837.323934,153.478871,197.637339,2,20,MOBILE_PHONE,iPhone 5c,Apple,iOS,10.3.3
2,24,6,6,4,8,14,176400,2,1,1415,272.071429,460.711400,61,1194036,256397.500000,449610.131423,0,38330,5587.714286,11579.649964,0.002059,0.635197,0.265700,0.276047,0,699,53.714286,185.871100,NaN,NaN,NaN,NaN,1,2,1.214286,0.425815,1,898,195.642857,333.989545,...,533.916739,61.000000,1349.193220,529.546165,534.058676,61.0,1349.193220,529.523340,534.022009,0.000000,1.750094,0.583365,1.010417,72.320755,89.000000,78.795755,8.115258,72.320755,89.000000,78.795755,8.115258,72.320755,89.000000,78.795755,8.115258,72.320755,89.000000,78.795755,8.115258,NaN,NaN,NaN,NaN,3,24,PRINTER,NaN,Hewlett Packard,HP ETHERNET MULTI-ENVIRONMENT,NaN
3,31,3,3,2,3,4,93600,2,1,20,9.500000,8.346656,221,2862,1658.000000,1216.304512,0,712,283.000000,347.844793,0.000000,0.427679,0.213839,0.302414,0,3,1.250000,1.500000,NaN,NaN,NaN,NaN,1,2,1.250000,0.500000,

In [0]:
final_df.to_csv('features_v2.csv')

In [14]:
final_df.shape

(76082, 131)

In [15]:
from google.colab import files
files.download("features_v2.csv")

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 53848, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [16]:
from google.colab import drive
drive.mount('drive')
final_df.to_csv('features_v2.csv')
!cp features_v2.csv drive/My\ Drive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive
